# Dense Model


## Section for Configurations and Imports

In [1]:
import os 
import tempfile

import numpy as np 
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
from tensorflow import keras

# MLflow dashboard
import mlflow
mlflow.set_tracking_uri('http://35.228.45.76:5000')
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='../../keys/mlflow-312506-8cfad529f4fd.json'

# Import data augmentation
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from augmentation.methods import *

In [2]:
import warnings
warnings.simplefilter('ignore')

In [3]:
random_state = 47
np.random.seed(random_state)

In [4]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss', 
    verbose=1,
    patience=10,
    mode='min',
    restore_best_weights=True)

# 1. Data Preparation

## 1. Load Prepared Dataset

In [5]:
data_path = '../../datasets/'
files = [
    'posenet_scored_all_1.csv',
    'posenet_scored_all_2.csv',
    'posenet_scored_all_3.csv',
    'posenet_scored_all_4.csv',
]
df = None

In [6]:
import pandas as pd
for file in files:
    try:
        dataset = pd.read_csv(data_path + file)
        dataset.drop(columns=['left_eye_x', 'left_eye_y', 'left_eye_score', 'right_eye_x', 'right_eye_y', 'right_eye_score', 
                                'left_ear_x', 'left_ear_y', 'left_ear_score', 'right_ear_x', 'right_ear_y', 'right_ear_score'],
                                inplace=True)
        dataset.rename(columns = {'nose_x': 'head_x', 'nose_y': 'head_y', 'nose_score': 'head_score'}, inplace = True)
        if df is None:
            df = dataset
        else:
            df = df.append(dataset, ignore_index=True)  
    except IOError as e:
        print('Error in reading file: ', e)
print(df.shape, '\n')
print(df.head())

(225092, 42) 

       head_x     head_y  head_score  left_shoulder_x  left_shoulder_y  \
0  421.555422  91.446746    0.995605       397.484966       131.001872   
1  421.555422  91.446746    0.995605       397.484966       131.001872   
2  421.555422  91.446746    0.995605       397.484966       131.001872   
3  421.555422  91.446746    0.995605       397.484966       131.001872   
4  421.555422  91.446746    0.995605       397.484966       131.001872   

   left_shoulder_score  right_shoulder_x  right_shoulder_y  \
0             0.998238        461.732625        132.726183   
1             0.998238        461.732625        132.726183   
2             0.998238        461.732625        132.726183   
3             0.998238        461.732625        132.726183   
4             0.998238        461.732625        132.726183   

   right_shoulder_score  left_elbow_x  ...  right_knee_score  left_ankle_x  \
0              0.999635    376.969123  ...          0.999779    416.909581   
1          

## 2. Split

In [7]:
X = df.drop(columns=['ConfidenceScore', 'GoodnessScore'])
y = df[['ConfidenceScore', 'GoodnessScore']]

In [8]:
input_mean = X.mean()
input_std = X.std()

X = (X - input_mean) / input_std

X.to_numpy()

array([[-0.44482851, -1.12477105,  0.26033375, ..., -1.4281565 ,
         0.24819995,  0.65616759],
       [-0.44482851, -1.12477105,  0.26033375, ..., -1.4281565 ,
         0.24819995,  0.65616759],
       [-0.44482851, -1.12477105,  0.26033375, ..., -1.4281565 ,
         0.24819995,  0.65616759],
       ...,
       [-4.36780409, -2.26515688, -6.07595562, ...,  1.08250379,
        -0.95094489, -6.23234963],
       [-4.36780409, -2.26515688, -6.07595562, ...,  1.08250379,
        -0.95094489, -6.23234963],
       [-4.36780409, -2.26515688, -6.07595562, ...,  1.08250379,
        -0.95094489, -6.23234963]])

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=random_state)

In [10]:
output_mean = y_train.mean()
output_std = y_train.std()

y_train = (y_train - output_mean) / output_std

y_train.to_numpy()

array([[ 0.04032739,  0.57164717],
       [-0.69074802, -0.41269108],
       [-1.43648694, -1.39702934],
       ...,
       [ 0.74555045,  0.57164717],
       [ 0.40349968, -0.41269108],
       [ 0.40210794,  0.57164717]])

In [11]:
# from sklearn.preprocessing import StandardScaler

# X_scaler = StandardScaler()
# X_train = X_scaler.fit_transform(X_train)
# X_test = X_scaler.transform(X_test)
# y_scaler = StandardScaler()
# y_train = y_scaler.fit_transform(y_train)

print('Training features shape:', X_train.shape)
print('Training labels shape:', y_train.shape, '\n')

print('Test features shape:', X_test.shape)
print('Test labels shape:', y_test.shape)

Training features shape: (202582, 40)
Training labels shape: (202582, 2) 

Test features shape: (22510, 40)
Test labels shape: (22510, 2)


## 3. Define the Model

### 3.1 Configurations

In [12]:
EPOCHS = 500
BATCH_SIZE = 150

input_dim = X_train.shape[1]
output_size = 2

units = 64
activation = 'relu'
#kernel_initializer = 'he_uniform'
kernel_initializer = 'normal'
#output_activation = 'sigmoid'
#output_activation = 'sigmoid'
output_activation = 'linear'
#optimizer = 'SGD'
#optimizer = 'RMSprop'
optimizer = 'Adam'
learning_rate = 1e-3

### 3.2 Model Architecture

In [13]:
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense

METRICS = [
    MeanSquaredError(name="mse", dtype=None),
    MeanAbsoluteError(name="mae", dtype=None),
    RootMeanSquaredError(name="rmse", dtype=None),
]

def make_model(input_dim, loss, optimizer, learning_rate=0.001, metrics=METRICS):
    #MODEL 3 - Champ
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, kernel_initializer=kernel_initializer, activation=activation))
    model.add(Dense(64, kernel_initializer=kernel_initializer, activation=activation))
    model.add(Dense(64, kernel_initializer=kernel_initializer, activation=activation))
    model.add(Dense(output_size, activation=output_activation))
    #http://35.228.45.76:5000/#/experiments/0/runs/0b6786ed5d8f4860a817b4d514aa4de5
    
    #MODEL 2
    #model.add(Dense(128, input_dim=input_dim, kernel_initializer=kernel_initializer, activation=activation))
    #model.add(Dense(64, kernel_initializer=kernel_initializer, activation=activation))
    #model.add(Dense(32, kernel_initializer=kernel_initializer, activation=activation))
    #model.add(Dense(8, kernel_initializer=kernel_initializer, activation=activation))
    #model.add(Dense(output_size, activation=output_activation))
    #http://35.228.45.76:5000/#/experiments/0/runs/12172f32a5d64cbfbd906c9d486cb90e
    
    #MODEL 1
    #model.add(Dense(64, input_dim=input_dim, kernel_initializer=kernel_initializer, activation=activation))
    #model.add(Dense(64, kernel_initializer=kernel_initializer, activation=activation))
    #model.add(Dense(32, kernel_initializer=kernel_initializer, activation=activation))
    #model.add(Dense(8, kernel_initializer=kernel_initializer, activation=activation))
    #model.add(Dense(output_size, activation=output_activation))
    #http://35.228.45.76:5000/#/experiments/0/runs/b7f73a3ba8944e139b5280c0e13cf505
    
    
    optimizer = tf.keras.optimizers.get(optimizer)
    optimizer.learning_rate.assign(learning_rate)

    model.compile(
      optimizer=optimizer,
      loss=loss,
      metrics=metrics)

    print(model.summary())

    return model

## 4. MLflow Experimentation

### 4.1 Configurations

In [14]:
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec
input_schema = Schema([
    ColSpec("double", "head_x"),
    ColSpec("double", "head_y"),
    ColSpec("double", "head_score"),
    ColSpec("double", "left_shoulder_x"),
    ColSpec("double", "left_shoulder_y"),
    ColSpec("double", "left_shoulder_score"),
    ColSpec("double", "right_shoulder_x"),
    ColSpec("double", "right_shoulder_y"),
    ColSpec("double", "right_shoulder_score"),
    ColSpec("double", "left_elbow_x"),
    ColSpec("double", "left_elbow_y"),
    ColSpec("double", "left_elbow_score"),
    ColSpec("double", "right_elbow_x"),
    ColSpec("double", "right_elbow_y"),
    ColSpec("double", "right_elbow_score"),
    ColSpec("double", "left_wrist_x"),
    ColSpec("double", "left_wrist_y"),
    ColSpec("double", "left_wrist_score"),
    ColSpec("double", "right_wrist_x"),
    ColSpec("double", "right_wrist_y"),
    ColSpec("double", "right_wrist_score"),
    ColSpec("double", "left_hip_x"),
    ColSpec("double", "left_hip_y"),
    ColSpec("double", "left_hip_score"),
    ColSpec("double", "right_hip_x"),
    ColSpec("double", "right_hip_y"),
    ColSpec("double", "right_hip_score"),
    ColSpec("double", "left_knee_x"),
    ColSpec("double", "left_knee_y"),
    ColSpec("double", "left_knee_score"),
    ColSpec("double", "right_knee_x"),
    ColSpec("double", "right_knee_y"),
    ColSpec("double", "right_knee_score"),
    ColSpec("double", "left_ankle_x"),
    ColSpec("double", "left_ankle_y"),
    ColSpec("double", "left_ankle_score"),
    ColSpec("double", "right_ankle_x"),
    ColSpec("double", "right_ankle_y"),
    ColSpec("double", "right_ankle_score"),
    ColSpec("double", "score")
])
output_schema = Schema([
    ColSpec("double", "ConfidenceScore"),
    ColSpec("double", "GoodnessScore"),
])
signature = ModelSignature(inputs=input_schema, outputs=output_schema)

### 4.2 Helper functions

In [15]:
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

def plot_train_history(history, title):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(loss))
    plt.figure()
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title(title)
    plt.legend()
    plt.show()

In [16]:
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error

def eval_metrics(actual, pred):
    mse = mean_squared_error(actual, pred)
    msa = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    variance = explained_variance_score(actual, pred)
    return mse, msa, r2, variance

### 4.3 Start Run

In [19]:
model_name = 'dense_model_confidence_goodness'

with mlflow.start_run(run_name=model_name) as run:

    run_id = run.info.run_id

    model = make_model(metrics=METRICS, 
                        loss='mean_squared_error', 
                        optimizer=optimizer, 
                        learning_rate=learning_rate, 
                        input_dim=input_dim)
    
    with tf.device('/CPU:0'):
        history = model.fit(
            x=X_train, 
            y=y_train, 
            shuffle=True, 
            epochs=EPOCHS, 
            verbose=1,
            batch_size=BATCH_SIZE,
            validation_split=0.2,
            callbacks=[early_stopping])

        # Plot training history
    plot_train_history(history, 'Baseline Training and validation loss')
    plt.savefig("training_history.jpg")
    mlflow.log_artifact("training_history.jpg")
    plt.show()

    # Log model parameters
    mlflow.log_param("activation", activation)
    mlflow.log_param("kernel initializer", kernel_initializer)
    mlflow.log_param("output activation", output_activation)
    mlflow.log_param("optimizer", optimizer)
    mlflow.log_param("learning rate", learning_rate)
    mlflow.log_param("batch size", BATCH_SIZE)
    mlflow.log_param("epochs", early_stopping.stopped_epoch)
    mlflow.log_param("input_mean", input_mean)
    mlflow.log_param("input_std", input_std)
    mlflow.log_param("output_mean", output_mean)
    mlflow.log_param("output_std", output_std)

    # Log model performance
    process_time = [None] * 10
    
    for i in range(10):
        start_time = time.time()
        predictions = model.predict(X_test)
        end_time = time.time()
        process_time[i] = (end_time - start_time) * 1000
    
    predictions = (predictions * output_std.to_numpy()) + output_mean.to_numpy() 
    (mse, mae, r2, variance) = eval_metrics(y_test, predictions)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("R-squared", r2)
    mlflow.log_metric("variance", variance)
    mlflow.log_param("total params", model.count_params())
    mlflow.log_metric("process time", np.mean(process_time)) 

    # Log model and scaler(s)
    mlflow.keras.log_model(model, model_name, signature=signature)
    #mlflow.sklearn.log_model(X_scaler, 'InputScaler')
    #mlflow.sklearn.log_model(y_scaler, 'OutputScaler')
    
    # Print metrics
    print('MSE: ', mse)
    print('MAE: ', mae)
    print('R-Squared: ', r2)
    print('Explained Variance Score: ', variance)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 128)               5248      
_________________________________________________________________
dense_9 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_10 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 130       
Total params: 17,794
Trainable params: 17,794
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
1081/1081 [==============================] - 5s 4ms/step - loss: 0.3223 - mse: 0.2653 - mae: 0.2977 - rmse: 0.5151 - val_loss: 0.2488 - val_mse: 0.2488 - val_mae: 0.2911 - val_rmse: 0.4988
Epoch 2/500
1081/1081 [===================

KeyboardInterrupt: 

## 5. Register Model

In [16]:
# model_uri = "runs:/{}/{}".format(run.info.run_id, model_name)
# mv = mlflow.register_model(model_uri, model_name)
# print("Name: {}".format(mv.name))
# print("Version: {}".format(mv.version))

Registered model 'dense_model_confidence_goodness' already exists. Creating a new version of this model...
2021/05/25 20:32:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: dense_model_confidence_goodness, version 22


Name: dense_model_confidence_goodness
Version: 22


Created version '22' of model 'dense_model_confidence_goodness'.
